In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import time

# Load pre-trained network for human pose estimation
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")

# Set input dimensions and threshold for confidence level
inWidth = 368  # Width for the input image
inHeight = 368 # Height for the input image
thr = 0.2      # Confidence threshold to detect body parts

# Define the body parts that the model can detect
BODY_PARTS = {"Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
              "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
              "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
              "LEye": 15}

# Define pairs of body parts that will be connected to form the skeleton
POSE_PAIRS = [["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
              ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
              ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
              ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"], ["Nose", "LEye"]]

# Function that does the actual pose estimation on each frame of video
def pos_estimation(frame):
    frameWidth = frame.shape[1] # Get the width of the frame
    frameHeight = frame.shape[0] # Get the height of the frame

    # Preprocess image to pass to the neural network
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))

    out = net.forward() # Forward pass through the network
    out = out[:, :19, :, :] # Limit output to first 19 body parts, rest not needed

    assert (len(BODY_PARTS) <= out.shape[1]) # Ensure that the output contains expected body parts

    points = [] # Store coordinates of detected body parts

    for i in range(len(BODY_PARTS)):
        heatMap = out[0, i, :, :] # Get the heatmap for each body part

        # Find the max confidence point for each body part
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3] # Calculate x-coordinate of the point
        y = (frameHeight * point[1]) / out.shape[2] # Calculate y-coordinate of the point

        points.append((int(x), int(y)) if conf > thr else None) # Store point if confidence is greater than threshold

    # Draw lines between connected body parts
    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert (partFrom in BODY_PARTS) # Make sure body part is defined
        assert (partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            # Draw a line between two body parts
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            # Draw circles on body joints
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    # Measure processing time and display it on the frame
    t, _ = net.getPerfProfile() # Performance profile of network
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    return frame # Return the frame with pose estimation drawn

# Start capturing video from the webcam
cap = cv.VideoCapture(0)

start_time = time.time() # Keep track of time
elapsed_time = 0 # Initialize elapsed time
while True:
    ret, frame = cap.read() # Read frame from the webcam
    if not ret: # If no frame, exit the loop
        break

    # Perform pose estimation on the current frame
    estimated_frame = pos_estimation(frame)

    # Display the frame with pose estimation
    cv.imshow('Pose Estimation', estimated_frame)

    # Check if the 'p' key is pressed, if so, stop the loop
    if cv.waitKey(1) & 0xFF == ord('p'):
        break

# Release the webcam and close any open windows
cap.release()
cv.destroyAllWindows()


In [2]:
pip install matplotlib



Note: you may need to restart the kernel to use updated packages.
